# Montu Python
## Astronomical ephemerides for the Ancient World
## Time: Basic functionalities


In [1]:
import montu
print(montu.version)

allstars = montu.Stars()

%load_ext autoreload
%autoreload 2

Running MontuPython version 0.9.6
0.9.6


## Aldebaran declination and Senenmut temple

In [72]:
Aldebaran = allstars.get_stars(ProperName = 'Aldebaran')

In [73]:
mtime = montu.Time('BCE1472-01-01 00:00:00')
senenmut = montu.Observer(lon=32, lat=26, height=0.089)
print(mtime.get_readable())

Montu Time Object:
-------------------------- 
Readable:
    Date in proleptic UTC: -1471-01-01 00:00:00.0000
    Date in mixed UTC: -1471-01-14 00:00:00
    Date in SPICE format: 1472 B.C. 01-01 00:00:00.00
    Date in caniucular format: hrw 1310-I-Peret-22
    Components: [-1, 1471, 1, 1, 0, 0, 0, 0]
Objects:
    Date in datetime64 format: -1471-01-01T00:00:00.000000
    Date in PyPlanet Epoch: 1183788.5
    Date in PyEphem Epoch: -1472/1/14 00:00:00
General:
    Is bce: True
    Is Julian: True
Uniform scales:
    Terrestrial time:
        tt: -109534126961.8
        jtd: 1183788.9009051
        htd: 478291.40090509993
    UTC time:
        et: -109534161600.0
        jed: 1183788.5
        hed: 478291.0
    Delta-t = TT - UTC = 34638.2



Location of Aldebaran in the sky:

In [74]:
Aldebaran.where_in_sky(at=mtime,observer=senenmut)

,MN,HD,HR,HIP,Gl,Name,OtherDesignations,ProperName,Bayer,Flamsteed,...,tt,jed,RAJ2000t,DecJ2000t,RAEpoch,DecEpoch,HA,az,el,zen
14,15,29139,1457,21421,Gl 171.1A,Aldebaran,87 Tau/87Alp Tau/Aldebaran/Gl 171.1A/HD 29139/...,Aldebaran,α Tau,87 Tau,...,-1.095341e+11,1183788.5,4.594642,16.691872,1.472892,3.293294,7.332279,282.103714,-16.348674,106.348674


In [75]:
montu.Util.dec2hex(Aldebaran.data.DecEpoch.iloc[0]), montu.Util.dec2hex(Aldebaran.data.DecJ2000.iloc[0])

('03:17:35.858', '16:30:33.484')

This confirm the fact that Aldebaran was at 3 degrees of declination.

In [76]:
Aldebaran.data.RAEpoch.iloc[0],Aldebaran.data.DecEpoch.iloc[0]

(1.4728921279394127, 3.2932937831486404)

Let's test that we can invert from az, el to RA, Dec

In [78]:
az, el = montu.Util.where_in_sky(Aldebaran.data.RAEpoch.iloc[0],Aldebaran.data.DecEpoch.iloc[0],at=mtime,observer=senenmut)
az, el

7.332278661086613 8.805170789026025


(282.10371441686937, -16.348674434537937)

In [79]:
montu.Util.where_in_equator(az=az,el=el,at=mtime,observer=senenmut)

(1.4728921279394118, 3.293293783148654)

Which is a perfect inversion

Now let's see what is the declination of a point with the azimuth and elevation of the entrance to the Senenmut temple:

In [80]:
site = montu.Observer(lon=32,lat=25.75,height=0)
montu.Util.where_in_equator(az=99,el=25,at=mtime.add(10000),observer=site)

(15.837783216950731, 3.204820559499014)

Let's now calculate which stars are in this declination, first we need to precess the position of all starrs:

In [42]:
allstars.where_in_space(at=mtime,inplace=True)

Now select the stars with a declination in the range:

In [44]:
allstars.get_stars(DecEpoch=[3,3.5],Vmag=[-2,4])

4 star(s):
|     |   MN |     HD |   HR |   HIP | Gl        | Name           | OtherDesignations                                                                     | ProperName     | Bayer   | Flamsteed   | Constellation   |   RAJ2000 |   DecJ2000 |    pmRA |   pmDec |   RadVel |   Distance |   Vmag |   Vmag_min |   Vmag_max |   B-V | SpType   |   Luminosity |   XJ2000 |   YJ2000 |    ZJ2000 |    VXJ2000 |    VYJ2000 |    VZJ2000 |   Primary | MultipleID   |   IsMultiple |   IsVariable |           tt |         jed |   RAJ2000t |   DecJ2000t |   RAEpoch |   DecEpoch |
|-----|------|--------|------|-------|-----------|----------------|---------------------------------------------------------------------------------------|----------------|---------|-------------|-----------------|-----------|------------|---------|---------|----------|------------|--------|------------|------------|-------|----------|--------------|----------|----------|-----------|------------|------------|------------|

That confirms that Aldebaran is the star.

## Date of the winter solstice in times of Senenmut

Let's compute first the date of the seasons at the interesting year:

In [63]:
sun = montu.Sun()
mtime=montu.Time('bce1479-01-01 00:00:00')
ss = sun.next_seasons(at=mtime)
for s in ss:
    stime = montu.Time(s,format='jd').get_readable()
    print(stime.readable.datespice,stime.rebadable.datecan)

1479 B.C. 03-21 15:22:19.700000 hrw 1303-IV-Peret-9
1479 B.C. 06-23 22:37:50.300000 hrw 1303-III-Shemu-13
1479 B.C. 09-23 03:46:48.00 hrw 1304-II-Akhet-10
1479 B.C. 12-20 12:51:54.00 hrw 1304-I-Peret-8


This confirms that the winter solstice is in I-Peret, but at day 8 and not day 6 as in Tito's thesis.

## Azimut of the sun at sunrise during winter solstice

We will investigata the claim that at winter solstice the azimut of the sun is different from the orientation of the Montuhotep-II temple:

In [83]:
sun = montu.Sun()
mtime=montu.Time('bce2050-01-01 00:00:00')
ss = sun.next_seasons(at=mtime)
mtime_winter = montu.Time(ss[-1],format='jd').get_readable()
mtime_winter

Time('-2049-12-20 17:16:12.3000'/'-2048-01-06 17:16:16'/'hrw 732-IV-Shemu-24'/JED 973031.2195868/JTD 973031.7734826)

Let's compute the position of the sun at the day of the winter solstice:

In [88]:
sun.condition

{'tt': -127743542771, 'jed': 973031.2195868, 'Name': 'Sun', 'ha': 7.262764128678882, 'Vmag': -26.8, 'rise_time': 973030.6996058573, 'rise_az': 116.52566369343715, 'set_time': 973031.1341546485, 'set_az': 243.47315787155992, 'transit_time': 973030.9168810539, 'transit_el': 40.084252634878375, 'elongation': 0.0, 'earth_distance': 0.9895421862602234, 'sun_distance': 0.0, 'is_circumpolar': False, 'is_neverup': False, 'angsize': 1939.54443359375, 'phase': 100.0, 'hlat': -5.9742855190132e-05, 'hlon': 90.00173737252734, 'hlong': 90.00173737252734}

In [92]:
sun.where_in_sky(at=mtime_winter,observer=senenmut)
sun.conditions_in_sky(at=mtime_winter,observer=senenmut)
dec = sun.position.DecEpoch
rise_az = sun.condition.rise_az
rise_az - 90, dec

(26.525663693437153, -23.933823942723922)

Obliquity for the time is:

In [102]:
from pyplanets.core.coordinates import true_obliquity as pyplanets_true_obliquity
epsilon = pyplanets_true_obliquity(mtime.obj_pyplanet)
epsilon

Angle(23.930957074243345)

According to sin law 

$$
\sin\Delta A = \sin\delta / \cos\epsilon
$$

In [109]:
import numpy as np
deltaA = np.arcsin(np.sin(abs(dec)*montu.DEG)/np.cos(senenmut.lat*montu.DEG))*montu.RAD
deltaA

26.831084780194047

The difference between $\Delta A$ and $\delta$ for small values of $\delta$ is:

$$
|\Delta A - \delta| = \delta\left|\frac{1}{\cos \varphi} - 1\right|
$$

In [114]:
abs(dec)*montu.DEG*(1/np.cos(senenmut.lat*montu.DEG) - 1)*montu.RAD

2.6949950189422474

## Montuemhat tomb, TT33

In [117]:
montuemhat = montu.Observer(lon=32.5,lat=25.75,height=0.116)
mtime = montu.Time('2021-05-26 07:00:00')
sun.where_in_sky(at=mtime,observer=montuemhat)
sun.position

{'tt': 675284472, 'jed': 2459360.7916666, 'Name': 'Sun', 'RAJ2000': 4.201918941681491, 'DecJ2000': 21.123846092039695, 'RAEpoch': 4.2222964942422, 'DecEpoch': 21.175976049635295, 'RAGeo': 4.222193002015081, 'DecGeo': 21.17636141039897, 'el': 51.585519385803636, 'az': 87.86147027534717}

In [118]:
montu.Util.dec2hex(sun.position.az)

'87:51:41.293'

In [124]:
montu.Util.where_in_equator(az=92.5,el=23.26,at=mtime,observer=montuemhat)

(13.34557131397417, 7.7857746514419635)

In [123]:
mtime = montu.Time('bce650-01-01 00:00:00')
Aldebaran.where_in_space(at=mtime)

,MN,HD,HR,HIP,Gl,Name,OtherDesignations,ProperName,Bayer,Flamsteed,...,Primary,MultipleID,IsMultiple,IsVariable,tt,jed,RAJ2000t,DecJ2000t,RAEpoch,DecEpoch
14,15,29139,1457,21421,Gl 171.1A,Aldebaran,87 Tau/87Alp Tau/Aldebaran/Gl 171.1A/HD 29139/...,Aldebaran,α Tau,87 Tau,...,21368,Gl 171.1,1,1,-8.359436e+10,1484017.5,4.595597,16.648636,2.17739,7.363492


## Tutmosis III

In [126]:
montuemhat = montu.Observer(lon=32.5,lat=25.75,height=0.116)
mtime = montu.Time('bce1425-05-26 07:00:00')
montu.Util.where_in_equator(az=350,el=40,at=mtime,observer=montuemhat)

(23.5645925212547, 73.48566392640559)

In [127]:
allstars.where_in_space(at=mtime,inplace=True)

In [128]:
allstars.get_stars(DecEpoch=[72,74],Vmag=[-2,4])

6 star(s):
|     |   MN |     HD |   HR |   HIP | Gl       | Name    | OtherDesignations                                                                 | ProperName   | Bayer   | Flamsteed   | Constellation   |   RAJ2000 |   DecJ2000 |    pmRA |   pmDec |   RadVel |   Distance |   Vmag |   Vmag_min |   Vmag_max |   B-V | SpType   |   Luminosity |   XJ2000 |     YJ2000 |   ZJ2000 |    VXJ2000 |    VYJ2000 |   VZJ2000 |   Primary | MultipleID   |   IsMultiple |   IsVariable |           tt |        jed |   RAJ2000t |   DecJ2000t |   RAEpoch |   DecEpoch |
|-----|------|--------|------|-------|----------|---------|-----------------------------------------------------------------------------------|--------------|---------|-------------|-----------------|-----------|------------|---------|---------|----------|------------|--------|------------|------------|-------|----------|--------------|----------|------------|----------|------------|------------|-----------|-----------|--------------|--

In [ ]:
sun.where_in_sky

## Possible solar alignment of TT353

In [150]:
montuemhat = montu.Observer(lon=32.5,lat=25.75,height=0.116)
mtime = montu.Time('bce1460-03-28 06:00:00')
sun.where_in_sky(at=mtime,observer=montuemhat)
sun.position.el

29.430363058046993

In [133]:
mtime = montu.Time('bce1479-01-01 06:00:00')
mtime

Time('-1478-01-01 06:00:00.0000'/'-1478-01-14 06:00:00'/'hrw 1303-I-Peret-20'/JED 1181231.75/JTD 1181232.1526134)

In [140]:
mtime = montu.Time('hrw1310-IV-Peret-2 00:00:00',calendar='caniucular')
mtime

Time('-1471-03-12 00:00:00.0000'/'-1471-03-25 00:00:00'/'hrw 1310-IV-Peret-2'/JED 1183858.5/JTD 1183858.9009051)